In [ ]:
def vic_data(datetime,requests,ET,pd):
    
    now = datetime.datetime.now()
    now = now.strftime('%Y%m%d-%H%M%S')
    
    vic_fire_data = 'data/vic_data/vic_fire_data_'+now+'.xml'

    URL = "https://data.emergency.vic.gov.au/Show?pageId=getIncidentRSS"

    response = requests.get(URL)
    with open(vic_fire_data , 'wb') as file:
        file.write(response.content)

    #parse through the data
    mytree = ET.parse(vic_fire_data) 
    myroot = mytree.getroot()

    all_desc_list =[]
    for all_description in myroot.findall("./channel/item/description"):
        all_desc_list.append(all_description.text)

    clean_1 = []
    for string in all_desc_list:
        clean_1.append(string.replace('</strong>',' '))

    clean_2 = []
    for string in clean_1:
        clean_2.append(string.replace('<strong>',' '))

    clean_3 = []
    for string in clean_2:
        clean_3.append(string.replace('<br>',' '))

    clean_data = []
    for entry in clean_3:
        clean_data.append(entry.split())

    #function to get lat co_ords
    vic_latitude_list = []
    def vic_lat_co_ordinates(data):
        for entry in clean_data:
            vic_latitude_list.append(entry[-3])
        return vic_latitude_list 
    vic_lat_co_ordinates(clean_data)

    #function to get long co_ords
    vic_longitude_list = []
    def vic_long_co_ordinates(data):
        for entry in clean_data:
            vic_longitude_list.append(entry[-1])
        return vic_longitude_list    
    vic_long_co_ordinates(clean_data) 

    #add todays date as report date
    report_date = datetime.date.today()
    report_date = report_date.strftime('%Y-%m-%d')
    
    vic_df = pd.DataFrame({
        'Latitude': vic_latitude_list,
        'Longitude': vic_longitude_list,
        'State': 'VIC',
        'Report Date': report_date
    })                         

    #append to the saved fire_data csv file
    vic_df.to_csv("data/fire_data.csv", mode = 'a', index = False , header = False)

    #remove duplicate values
    fire_data_df = pd.read_csv('data/fire_data.csv')

    #drop the duplicates out of the data frame
    distinct_fire_data = fire_data_df.drop_duplicates()

    #create a new csv file with the dropped duplicate values
    distinct_fire_data.to_csv('data/fire_data_distinct.csv',index = False)
    
